> # Notebook para el EDA

In [30]:
# Librerias
import pandas as pd
import yaml
import numpy as np

# Cargando Yaml
with open("../config.yaml", "r") as file:
    config = yaml.safe_load(file)

In [2]:
df = pd.read_excel(r"../Data/Base de datos MITOS.xlsx")

>> ## Exploración y Valores Nulos

>>>### Parte I

Dado a que la investigación fue realizada dividiendo a los individuos en dos grupos, para aplicar 2 tratamientos distintos en 2 tiempos, revisaremos las siguientes columnas:

- visita
- grupo
- periodo
- tiempo
- tratamiento

Todos los individuos deberían de tener 4 visitas y un grupo, periodo y tiempo asignado. Las llamaremos **columnas identificadoras** pues únicamente sirven para identificar a los individuos así como a su tiempo/fase en el experimento.

In [10]:
# Contando registros por persona
df['visita'].value_counts().reset_index()

,visita,count
0,1,42
1,2,42
2,3,42
3,4,42


In [4]:
col_identificadoras = ['grupo', 'periodo', 'tiempo', 'tratamiento']

df[col_identificadoras].isna().sum()

grupo          6
periodo        8
tiempo         8
tratamiento    6
dtype: int64

In [5]:
df[df[col_identificadoras].isna().any(axis=1)]['id'].unique()

array([42, 46])

Podemos observar que los IDs 42 y 46 tienen valores nulos en las columnas identificadoras.

Dada la consulta con los compañeros de España, se decidió darles el siguiente tratamiento: **excluir a los individuos**

>>>### Parte II

Debido a que la base de datos se encuentra en un formato wide, lo transformaremos a uno deep para identificar si algún individuo no tiene alguna visita.

In [6]:
df_deep = df.copy()

# Eliminando columnas identificadoras para no confundirnos
df_deep.drop(columns=col_identificadoras,inplace=True)

# Transformando en formato deep
df_deep = pd.melt(df_deep, id_vars=['id', 'visita'], value_name='valor')

# Agrupando para encontrar nulos individuos sin ningun valor en una visita
df_visita_nula = df_deep.groupby(['id', 'visita'])['valor'].apply(lambda x: x.isnull().mean()).reset_index()

In [7]:
# Filtrando a los IDs que tengan visitas totalmente vacías
df_visita_nula[df_visita_nula.valor == 1]

,id,visita,valor
165,42,2,1.0
166,42,3,1.0
167,42,4,1.0
181,46,2,1.0
182,46,3,1.0
183,46,4,1.0


Podemos identificar a los individuos 8 y 26 que no tienen ningún valor en la cuarta visita.

Dada la consulta con los compañeros de España, se decidió darles el siguiente tratamiento: **excluir a los individuos**

In [8]:
# Excluyendo a los individuos que los compañeros nos indican, por ausencia total de información relevante.
df = df[~df['id'].isin([42, 46, 8, 26, 30])]

# Recalculando DataFrame en formato Deep
col_identificadoras = ['visita','grupo', 'periodo', 'tiempo', 'tratamiento']
df_deep = df.copy()


# Transformando en formato deep
df_deep = pd.melt(df_deep, id_vars=['id', 'visita'], value_name='valor')

In [ ]:
nulls_count = df.isna().sum().reset_index()

nulls_count.columns = ['Variable','Null_Count']

In [20]:
nulls_count[nulls_count['Null_Count'] > 0]

,Variable,Null_Count
6,frec_alcohol_audit,128
7,puntaje_audit,128
8,Palabras_moca,126
9,memoria_moca,126
10,puntaje_moca,126
...,...,...
647,delta_d2_con_pd,84
648,delta_d2_vt_pd,84
649,delta_fn_nombre,84
650,delta_fn_score,84


>> ## Tratamiento de Variables y Valores Nulos

Tenemos **309** variables con valores nulos.

Dada nuestra conversación con los compañeros de Barcelona, nos juntamos para definir que tratamiento darles a los valores nulos de cada variable.

Aprovechando que colaboramos con expertos en el tema, pudimos tomar una decisión precisa para cada una de ellas.
Se optó por eliminar ciertas variables que no estaban muy completas o que ellos no consideraron relevantes para el estudio.

Algunas otras se les dará tratamiento especial y en otras llenaremos los valores nulos con la técnica que se mencionará a continuación.

>>> ### Tratamiento **Especial**

Estas variables, recibimos instrucciones especiales para tratar con ellas o con los valores nulos, a continuación el detalle por variable.

In [32]:
# Dado que esta columna solamente tiene una observación por Id, la rellenaremos con la fecha de nacimiento del mismo individuo.
df['f_nac'] = df['f_nac'].ffill()

# Posteriormente, se nos indicó calcular la edad al momento de la visita. Se calculará a 2 decimales.
df['edad'] = ((pd.to_datetime(df['fecha_visita']) - pd.to_datetime(df['f_nac'])).dt.days / 365).round(2)

>> ## Estadísticas de Variables

Definiremos los grupos de variables según lo establecido por los estudiantes del PHD en Nutrición.

- Demografía/estilo de vida (23 variables)
- Antropometría y signos vitales (38)
- Laboratorio clínico (41)
- Uroanálisis (24)
- Actigrafía y sueño objetivo (17)
- Cuestionarios de sueño PGH/ISI (40)
- Salud mental HADS (3)
- Calidad de vida WHOQOL‑BREF (38)
- Actividad física IPAQ (9)
- Cognición —MoCA, D2, WCST y funciones nominales— (31)
- Frecuencia alimentaria (FFQ) (181 ítems, incluyendo preparaciones de alimentos)
- Consumo/preparación específica de tomate (15)
- Carotenoides/licopeno (11)
- Marcadores neurotróficos y de envejecimiento (5)

En conjunto, el contenido permite relacionar **exposición dietética (tomate y licopeno)** con **biomarcadores y resultados cognitivos**.
